# Lab 5 - Parcel Feature Extraction

Next, we will illustrate the construction of features related to our main task: finding the relationship between property development and water quality over time.  In a previous lab, you identified lakes for which we have complete information for the years from 2004 to 2015.  In this lab, we will

[Original Data and variable information](https://gisdata.mn.gov/organization/us-mn-state-metrogis?q=Metro+Regional+Parcel+Dataset&sort=score+desc%2C+metadata_modified+desc)

## Problem 1 - Feature construction

**Overview.** Remember that our target output file will have one row per year-lake combination.  To attach property information, we will need to group and aggregate the parcel data to create features for each lake-year combination.  When grouping the data, be sure to maintain the variables needed to join to the water quality data, namely the lake ID and year.  Since we are looking at tracking property development/change over time, we will want to generate features tracking

* Number of properties close to each lake,
* Summaries of the value of properties close to each lake,
* Aggregations on the size and type of the properties, and
* Other features that might impact water quality.
    
#### Task 1. Understanding parcel variables

Before we can construct features, we need to make sure we understand the parcel data.  The metro parcel data is provided by the State of Minnesota and the meta data can be found online.  For example, searching for *metro parcel 2014* lead to [this site](https://geo.btaa.org/catalog/304cf3d8-a53b-4ea9-b02a-f550bd68e320).  Clicking on the *Meta data* button in the top left, brought up more information.  Clicking *Download* opened in this meta data [in a separate page](https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_parcels_2014/metadata/metadata.html)

Look through the **Section 4: Attributes** and identify variables that might impact the water quality of near-by lakes.

> <font color="orange"> Your thoughts here </font>

Land-use variables like USE1_DESC–USE4_DESC, DWELL_TYPE, and HOME_STYLE can tell us how the land is developed and how much runoff it might produce
Features like FIN_SQ_FT, GARAGESQFT, NUM_UNITS, and ACRES_POLY can help us identify how nutrient and stormwater flow into lakes
Attributes like GREEN_ACRE, OPEN_SPACE, and AG_PRESERV are also good because preserved land vs agricultural land can have very different impacts on water quality

### Task 2. Feature Brainstorming

Our objective is to build a feature table with one row per lake-year, using grouped summary statistics. Here are effective strategies for feature construction:

1. **Numerical summaries:** Calculate group-level statistics (mean, median, standard deviation, IQR, etc.) for numeric variables.
2. **Categorical summaries:** For text data, consider:
   - **Success rates:** Compute proportions for binary variables (e.g., percent of homes with basements).
   - **Label cleaning:** Review and standardize unique labels to remove duplicates or inconsistencies.
   - **Broader categories:** Recode variables with many rare categories into a smaller, more meaningful set.
   - **Indicator columns:** Create indicator variables and aggregate them to show presence/absence or proportions (e.g., count of each property use type).

Review the variables you identified earlier and outline a feature construction strategy for each.

> <font color="orange"> Your thoughts here </font>

For the variables I identified earlier, I would create lake-year features by summarizing what the surrounding parcels look like
For numeric variables (like FIN_SQ_FT, NUM_UNITS, GARAGESQFT, ACRES_POLY), I’d use group statistics such as mean or median to show development intensity 
For categorical variables (like USE1_DESC, DWELL_TYPE, GREEN_ACRE), I’d create proportions or indicator counts to show how common each land-use type is around each lake

### Task 4. Initial querying with filter and select

First, you should build a query that filters the parcel data to 
1. only include parcels within 1600 feet of the lakes we are studying, and 
2. only for the lakes with complete information.  

You should also select only the columns you will need for feature construction and joining to the water quality data.

In [2]:
import polars as pl

In [3]:
import polars as pl

dnr_id_site_numbers = (
    pl.scan_parquet("data/lakes_processed/water_quality_by_year.parquet")
    .select("DNR_ID_Site_Number")
    .unique()
    .collect()
    .to_series()
    .to_list()
)


In [4]:
xref = pl.scan_parquet("data/xref_partitioned.parquet")


In [5]:
xref = pl.scan_parquet("data/xref_partitioned.parquet")

parcel_query = (
    pl.scan_parquet("data/lakes_processed/parcel_combined.parquet")
    .join(xref, on=["centroid_lat", "centroid_long"], how="inner")
    .filter(
        pl.col("distance_category").is_in([
            "within_500_meters",
            "between_501_1600_meters",
        ])
    )
    .filter(
        pl.col("Monit_MAP_CODE1").is_in(dnr_id_site_numbers)
    )
    .select([
        "Year",
        "Monit_MAP_CODE1",
        "distance_category",
        "EMV_TOTAL",
        "EMV_LAND",
        "EMV_BLDG",
        "DWELL_TYPE",
        "HOME_STYLE",
        "FIN_SQ_FT",
        "NUM_UNITS",
        "GREEN_ACRE",
        "OPEN_SPACE",
        "AG_PRESERV",
        "GARAGE",
        "BASEMENT",
        "centroid_lat",
        "centroid_long",
        "TAX_EXEMPT",
    ])
)




In [6]:
parcel_query.limit(10).collect()


Year,Monit_MAP_CODE1,distance_category,EMV_TOTAL,EMV_LAND,EMV_BLDG,DWELL_TYPE,HOME_STYLE,FIN_SQ_FT,NUM_UNITS,GREEN_ACRE,OPEN_SPACE,AG_PRESERV,GARAGE,BASEMENT,centroid_lat,centroid_long,TAX_EXEMPT
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2007""","""02000500-01""","""between_501_1600_meters""","""100""","""100""","""0""","""RESIDENTIAL SINGLE FAMILY""",null,"""0""","""0""",null,null,"""N""",null,null,"""45.19678""","""-93.10156""","""N"""
"""2007""","""02000500-01""","""between_501_1600_meters""","""100""","""100""","""0""","""RESIDENTIAL SINGLE FAMILY""",null,"""0""","""0""",null,null,"""N""",null,null,"""45.19678""","""-93.10156""","""N"""
"""2007""","""02000500-01""","""between_501_1600_meters""","""346700""","""89500""","""257200""","""COMMERCIAL / GOLF COURSE""","""Restaurant""","""3560""","""0""",null,null,"""N""",null,"""N""","""45.19652""","""-93.10094""","""N"""
"""2007""","""02000500-01""","""between_501_1600_meters""","""346700""","""89500""","""257200""","""COMMERCIAL / GOLF COURSE""","""Restaurant""","""3560""","""0""",null,null,"""N""",null,"""N""","""45.19652""","""-93.10094""","""N"""
"""2007""","""02000500-01""","""between_501_1600_meters""","""238700""","""86300""","""152400""","""RESIDENTIAL DUPLEXES, DOUBLE *""","""English duplex""","""1040""","""0""",null,null,"""N""",null,"""Y""","""45.19491""","""-93.10232""","""N"""
"""2007""","""02000500-01""","""between_501_1600_meters""","""238700""","""86300""","""152400""","""RESIDENTIAL DUPLEXES, DOUBLE *""","""English duplex""","""1040""","""0""",null,null,"""N""",null,"""Y""","""45.19491""","""-93.10232""","""N"""
"""2007""","""02000500-01""","""between_501_1600_meters""","""51400""","""30400""","""21000""","""COMMERCIAL / GOLF COURSE""","""Storage Warehouse""","""289""","""0""",null,null,"""N""",null,"""N""","""45.19479""","""-93.1003""","""N"""
"""2007""","""02000500-01""","""between_501_1600_meters""","""51400""","""30400""","""21000""","""COMMERCIAL / GOLF COURSE""","""Storage Warehouse""","""289""","""0""",null,null,"""N""",null,"""N""","""45.19479""","""-93.1003""","""N"""
"""2007""","""02000500-01""","""between_501_1600_meters""","""608700""","""97000""","""511700""","""COMMERCIAL / GOLF COURSE""","""Office Building""","""5766""","""0""",null,null,"""N""",null,"""N""","""45.19469""","""-93.10079""","""N"""


## Problem 2.  Numerical Summaries

Two important categories of property data involve the size (e.g., finished square footage) and value (e.g., accessed value and/or taxes paid).

**Tasks.** 

1. Identify 2-3 variables for each of these categories.
2. Write a query that computes the summary statistics for each of these variables for each lake-year.  
3. Write this summary table out to a CSV file named `parcel_numerical_summaries.csv`.  Again, you should partition by lake ID and year.

In [7]:
# Your code here
parcel_numerical_summaries = (
    parcel_query
    .with_columns([
        pl.col("FIN_SQ_FT").cast(pl.Float64, strict=False),
        pl.col("NUM_UNITS").cast(pl.Float64, strict=False),
        pl.col("EMV_TOTAL").cast(pl.Float64, strict=False),
        pl.col("EMV_LAND").cast(pl.Float64, strict=False),
        pl.col("EMV_BLDG").cast(pl.Float64, strict=False),
    ])
    .group_by(["Monit_MAP_CODE1", "Year"])
    .agg([
        pl.col("FIN_SQ_FT").mean().alias("mean_FIN_SQ_FT"),
        pl.col("FIN_SQ_FT").median().alias("median_FIN_SQ_FT"),
        pl.col("NUM_UNITS").mean().alias("mean_NUM_UNITS"),
        pl.col("NUM_UNITS").median().alias("median_NUM_UNITS"),
        pl.col("EMV_TOTAL").mean().alias("mean_EMV_TOTAL"),
        pl.col("EMV_TOTAL").median().alias("median_EMV_TOTAL"),
        pl.col("EMV_LAND").mean().alias("mean_EMV_LAND"),
        pl.col("EMV_LAND").median().alias("median_EMV_LAND"),
        pl.col("EMV_BLDG").mean().alias("mean_EMV_BLDG"),
        pl.col("EMV_BLDG").median().alias("median_EMV_BLDG"),
    ])
)

parcel_numerical_summaries.collect().write_csv(
    "data/lakes_processed/parcel_numerical_summaries.csv"
)



In [8]:
parcel_numerical_summaries.limit(10).collect()


Monit_MAP_CODE1,Year,mean_FIN_SQ_FT,median_FIN_SQ_FT,mean_NUM_UNITS,median_NUM_UNITS,mean_EMV_TOTAL,median_EMV_TOTAL,mean_EMV_LAND,median_EMV_LAND,mean_EMV_BLDG,median_EMV_BLDG
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""19002400-01""","""2009""",5301.211788,1548.0,1.760413,1.0,369728.762279,160000.0,78461.335953,33600.0,291267.426326,129700.0
"""10005200-01""","""2013""",1578.051118,1368.0,49.0,49.0,254461.767838,186300.0,130488.072417,66800.0,123973.695421,74900.0
"""19002601-01""","""2015""",2441.71159,2267.0,0.868104,1.0,296922.084456,268000.0,91684.204852,64700.0,205237.879605,195200.0
"""10000200-01""","""2005""",123.271698,0.0,null,null,452886.792453,394900.0,165477.358491,135000.0,287409.433962,278000.0
"""10001900-01""","""2008""",1705.020925,1749.0,null,null,352868.612892,282800.0,144924.232197,96200.0,207944.380695,178100.0
"""82008700-01""","""2006""",1139.486766,1158.0,null,null,312014.729574,232200.0,165199.539701,75900.0,146815.189873,150100.0
"""19002700-01""","""2008""",2161.535338,2162.0,0.918192,1.0,307159.388186,261800.0,119539.029536,70200.0,187620.35865,186800.0
"""19034800-01""","""2008""",1674.003403,1624.0,1.1909,1.0,228260.90507,198700.0,47382.715209,46800.0,180878.18986,155600.0
"""82005400-01""","""2014""",1005.334975,1100.0,0.651515,1.0,256865.517241,272700.0,137182.26601,140500.0,119683.251232,123900.0


In [9]:
import os
os.path.exists("data/lakes_processed/parcel_numerical_summaries.csv")


True

## Problem 3.  Simple categorical summaries.

In this part, you will create summary statistics for some of the simpler categorical variables.

**Binary variables.** There are two examples of binary variables, listed below.  You will need to compute the percent of `Yes` for each.

* GARAGE: Garage Y/N
* BASEMENT: Basement Y/N

**Other categorical variables.** There are a number of other categorical variables.  You need to select one of these variables, inspect/clean your variable as needed, create indicator variables for each resulting label, and compute summary statistics for each label.

* HOMESTEAD: Homestead Status
* TAX_EXEMPT: Tax Exempt Status 
* DWELL_TYPE: Dwelling Type 
* HOME_STYLE: Home Style
* HEATING: Heating type
* COOLING: Cooling type

**Tasks.**
Create a query that

1. Select one binary and two other categorical variables for feature construction,
2. Reads in the parcel data and selects the relevant columns (be sure to keep the lake ID and year),
3. Inspect unique labels and recode/clean as needed,
4. Create a literal column of ones, and
5. Pivot to get the counts of each label per lake-year (do this once per category).

Write this summary table out to a csv file named `parcel_categorical_summaries.csv`.  Again, you should partition by lake ID and year.

In [10]:
# Your code here
[
    parcel_query.select("GARAGE").unique().collect(),
    parcel_query.select("DWELL_TYPE").unique().collect(),
    parcel_query.select("TAX_EXEMPT").unique().collect()

]


[shape: (3, 1)
 ┌────────┐
 │ GARAGE │
 │ ---    │
 │ str    │
 ╞════════╡
 │ Y      │
 │ N      │
 │ null   │
 └────────┘,
 shape: (102, 1)
 ┌────────────────────────────────┐
 │ DWELL_TYPE                     │
 │ ---                            │
 │ str                            │
 ╞════════════════════════════════╡
 │ COMMERCIAL / GOLF COURSE, RES* │
 │ Single-Family / Owner Occupied │
 │ THEATER                        │
 │ AGRICULTURAL, AGRICULTURAL,    │
 │ RESIDENTIAL DUPLEXES, DOUBLE * │
 │ …                              │
 │ CHURCH                         │
 │ LUMBER STG                     │
 │ Double Bungalow/Duplex         │
 │ VET HSPTL                      │
 │ TWO FAMILY DWELLING            │
 └────────────────────────────────┘,
 shape: (3, 1)
 ┌────────────┐
 │ TAX_EXEMPT │
 │ ---        │
 │ str        │
 ╞════════════╡
 │ Y          │
 │ N          │
 │ null       │
 └────────────┘]

In [14]:
garage_summary = (
    parcel_query
    .with_columns(
        pl.col("GARAGE")
          .str.to_uppercase()
          .eq("Y")
          .cast(pl.Float64)
          .alias("garage_yes")
    )
    .group_by(["Monit_MAP_CODE1", "Year"])
    .agg(pl.col("garage_yes").mean().alias("pct_with_garage"))
    .collect()
)

tax_summary = (
    parcel_query
    .with_columns(
        pl.col("TAX_EXEMPT")
          .str.to_uppercase()
          .eq("Y")
          .cast(pl.Float64)
          .alias("tax_exempt_yes")
    )
    .group_by(["Monit_MAP_CODE1", "Year"])
    .agg(pl.col("tax_exempt_yes").mean().alias("pct_tax_exempt"))
    .collect()
)

home_summary = (
    parcel_query
    .with_columns(
        pl.col("HOME_STYLE").str.to_uppercase()
    )
    .select(["Monit_MAP_CODE1", "Year", "HOME_STYLE"])
    .with_columns(pl.lit(1).alias("count"))
    .group_by(["Monit_MAP_CODE1", "Year", "HOME_STYLE"])
    .agg(pl.col("count").sum())
    .collect()  
   .pivot(
    index=["Monit_MAP_CODE1", "Year"],
    on="HOME_STYLE",
    values="count",
)
    .fill_null(0)
)


In [15]:
parcel_categorical_summaries = (
    garage_summary
    .join(tax_summary, on=["Monit_MAP_CODE1", "Year"], how="left")
    .join(home_summary, on=["Monit_MAP_CODE1", "Year"], how="left")
)


In [16]:
parcel_categorical_summaries.write_csv(
    "data/lakes_processed/parcel_categorical_summaries.csv"
)


In [17]:
parcel_categorical_summaries.head()


Monit_MAP_CODE1,Year,pct_with_garage,pct_tax_exempt,null,1 3/4 STORY,BUNGALOW,1 STORY TOWNHOUSE,CONDO,MFD HOME (SINGLE),1-3/4 STRY,SPLIT LEVEL,SPLIT FOYER FRAME,1 STORY BRICK,SPLIT ENTRY,ONE STORY,ONE LEVEL,SPLIT LEVEL FRAME,NEIGHBORHOOD SHOPPING CTR,2 STORY TOWNHOUSE,2 STORY FRAME,1-1/4 STRY,TWO STORY,MULTI LEVEL,TWO+ STORY,COLONIAL,SPLIT/ENTRY,3-LVL SPLT,NONE,MODIFIED 2 STORY,1 STORY FRAME,1 STORY CONDO,1 1/2 STORY FRAME,SPLIT ENTY,SPLIT LEVL,BI-LEVEL,1 STORY,…,FIRE STATION,1 STORY A-FRAME,1 3/4 STRY,BAR/TAVERN,BI-LEVEL (TOTAL AREA),2 STORY,FIRE STATION (VOLUNTEER),E,1 3/4 STORY FINISHED,SPLIT-ENT,SPLIT-FOY,DOUBLE BUNGALOW - ON STORY,SPLIT/LEVEL,DISCOUNT STORE,4 LVL SPLT,TWIN HOME - SPIT LEVEL/ SPLIT*,RESTROOM BUILDING,L,DOUBLE,BANK,ONE AND 3/4 STORY,O,DOUBLE BUNGALOW - SPLIT FOYER,X,R,H,TWIN HOME,ELDERLY ASSIST. MULTI. RES.,RESTAURANT,2 STORY LOG-PINE,TWIN HOME - SPIT LEVEL/ SPLIT,FAST FOOD RESTAURANT,MFD HOME (DOUBLE),MINI-MART CONVENIENCE STORE W*,D,INDUSTRIALS LIGHT MFTG.,RELOCATABLE CLASSROOM
str,str,f64,f64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""02000500-01""","""2012""",0.703397,0.081193,352,0,0,0,0,0,0,336,0,0,0,552,0,0,16,0,0,0,316,0,0,0,0,0,0,0,0,0,0,0,0,602,0,…,4,0,0,8,0,0,0,0,8,0,0,4,0,12,0,8,12,0,0,4,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""19034800-01""","""2007""",null,0.013856,386,0,0,0,0,0,4,0,0,0,0,3108,502,0,0,0,0,6,3036,0,0,0,0,0,0,0,0,0,0,0,4126,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""27071100-01""","""2011""",1.0,0.023678,3372,0,0,0,352,0,0,404,0,0,460,0,0,0,0,0,0,0,0,0,0,348,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""70002600-01""","""2013""",null,0.025795,374,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,518,0,0,0,0,172,0,0,0,0,0,0,0,0,0,…,0,0,16,0,0,0,0,0,0,1450,68,0,0,0,312,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0
"""27003501-01""","""2005""",null,0.042959,5866,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Problem 4.  Join all the summaries.

Finally, you need to join all the summaries created above, along with the water quality summaries created in a previous lab, into one overall summary file.  Write the resulting table to a CSV file named `water_quality_and_parcel_summaries_2004_to_2015.csv`.

In [18]:
water_quality_summaries_complete = (
    pl.read_parquet("data/lakes_processed/water_quality_by_year.parquet")
)


In [23]:
water_q_df = water_quality_summaries_complete.with_columns(
    pl.col("Year").cast(pl.Utf8)
)


In [28]:
parcel_numerical_summaries = parcel_numerical_summaries.with_columns(
    pl.col("Year").cast(pl.Int32)
)

parcel_categorical_summaries = parcel_categorical_summaries.with_columns(
    pl.col("Year").cast(pl.Int32)
)


In [31]:
print(water_q_df.columns)
water_q_df.head()


['DNR_ID_Site_Number', 'Year', 'LAKE_NAME', 'avg_secchi_depth', 'avg_total_phosphorus']


DNR_ID_Site_Number,Year,LAKE_NAME,avg_secchi_depth,avg_total_phosphorus
str,i32,str,f64,f64
"""19003300-01""",2006,"""Earley Lake""",1.533333,0.045333
"""19044600-01""",2012,"""Lac Lavon Lake""",3.564286,0.026143
"""13005300-01""",2009,"""Big Comfort Lake""",1.954167,0.03225
"""82015900-01""",2007,"""Forest Lake""",1.314286,0.0335
"""27007000-01""",2005,"""Mitchell Lake""",1.4085,0.072286


In [32]:
parcel_num_df = pl.read_csv(
    "data/lakes_processed/parcel_numerical_summaries.csv"
)

parcel_cat_df = pl.read_csv(
    "data/lakes_processed/parcel_categorical_summaries.csv"
)

water_q_df = water_quality_summaries_complete

parcel_num_df = parcel_num_df.with_columns(
    pl.col("Year").cast(pl.Int32)
)

parcel_cat_df = parcel_cat_df.with_columns(
    pl.col("Year").cast(pl.Int32)
)

water_q_df = water_q_df.with_columns(
    pl.col("Year").cast(pl.Int32)
)

parcel_all = parcel_num_df.join(
    parcel_cat_df,
    on=["Monit_MAP_CODE1", "Year"],
    how="inner"
)

final_summary = (
    parcel_all
    .join(
        water_q_df,
        left_on=["Monit_MAP_CODE1", "Year"],
        right_on=["DNR_ID_Site_Number", "Year"],
        how="inner"
    )
)

final_summary.write_csv(
    "data/lakes_processed/water_quality_and_parcel_summaries_2004_to_2015.csv"
)


In [33]:
import os
os.path.exists("data/lakes_processed/water_quality_and_parcel_summaries_2004_to_2015.csv")


True

In [34]:
check = pl.read_csv("data/lakes_processed/water_quality_and_parcel_summaries_2004_to_2015.csv")
check.head()


Monit_MAP_CODE1,Year,mean_FIN_SQ_FT,median_FIN_SQ_FT,mean_NUM_UNITS,median_NUM_UNITS,mean_EMV_TOTAL,median_EMV_TOTAL,mean_EMV_LAND,median_EMV_LAND,mean_EMV_BLDG,median_EMV_BLDG,pct_with_garage,pct_tax_exempt,null,1 3/4 STORY,BUNGALOW,1 STORY TOWNHOUSE,CONDO,MFD HOME (SINGLE),1-3/4 STRY,SPLIT LEVEL,SPLIT FOYER FRAME,1 STORY BRICK,SPLIT ENTRY,ONE STORY,ONE LEVEL,SPLIT LEVEL FRAME,NEIGHBORHOOD SHOPPING CTR,2 STORY TOWNHOUSE,2 STORY FRAME,1-1/4 STRY,TWO STORY,MULTI LEVEL,TWO+ STORY,COLONIAL,SPLIT/ENTRY,…,BAR/TAVERN,BI-LEVEL (TOTAL AREA),2 STORY,FIRE STATION (VOLUNTEER),E,1 3/4 STORY FINISHED,SPLIT-ENT,SPLIT-FOY,DOUBLE BUNGALOW - ON STORY,SPLIT/LEVEL,DISCOUNT STORE,4 LVL SPLT,TWIN HOME - SPIT LEVEL/ SPLIT*,RESTROOM BUILDING,L,DOUBLE,BANK,ONE AND 3/4 STORY,O,DOUBLE BUNGALOW - SPLIT FOYER,X,R,H,TWIN HOME,ELDERLY ASSIST. MULTI. RES.,RESTAURANT,2 STORY LOG-PINE,TWIN HOME - SPIT LEVEL/ SPLIT,FAST FOOD RESTAURANT,MFD HOME (DOUBLE),MINI-MART CONVENIENCE STORE W*,D,INDUSTRIALS LIGHT MFTG.,RELOCATABLE CLASSROOM,LAKE_NAME,avg_secchi_depth,avg_total_phosphorus
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,f64,f64
"""02000500-01""",2012,1811.576636,1228.0,0.006628,0.0,200414.333057,175900.0,79295.443248,74300.0,121118.889809,109700.0,0.703397,0.081193,352,0,0,0,0,0,0,336,0,0,0,552,0,0,16,0,0,0,316,0,0,0,0,…,8,0,0,0,0,8,0,0,4,0,12,0,8,12,0,0,4,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""George Watch Lake""",0.359,0.2649
"""19034800-01""",2007,1633.298962,1612.0,1.184944,1.0,231077.50553,202200.0,46602.126936,46800.0,184475.378595,160900.0,null,0.013856,386,0,0,0,0,0,4,0,0,0,0,3108,502,0,0,0,0,6,3036,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Valley Lake""",1.428571,0.075714
"""27071100-01""",2011,935.985793,1008.0,null,null,347323.125493,211400.0,152447.750592,72350.0,194875.374901,137000.0,1.0,0.023678,3372,0,0,0,352,0,0,404,0,0,460,0,0,0,0,0,0,0,0,0,0,348,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Westwood Lake""",1.08375,0.033875
"""70002600-01""",2013,1204.324735,1196.0,null,null,314219.852602,219300.0,147346.79871,70000.0,166873.053892,132900.0,null,0.025795,374,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,518,0,0,0,0,…,0,0,0,0,0,0,1450,68,0,0,0,312,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,"""Lower Prior Lake""",3.436364,0.018909
"""27003501-01""",2005,0.0,0.0,null,null,346583.804978,238000.0,114696.454143,74000.0,231425.775656,159000.0,null,0.042959,5866,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Sweeney Lake""",1.43,0.0489


## Problem 5.  Put it all together

It is often useful to package all of the data constructions steps together in one convenient place.  Your last task is to

1. Gather all of your data construction code below.
    * You don't need to include exploratory code, e.g., exploring join mismatches; only the code necessary to combine, clean, and write your data.
2. Clean/refactor the code.
3. Be sure to display all important intermediate results.

In [ ]:
# Your code here.

In [ ]:
parcel_all = parcel_num_df.join(
    parcel_cat_df,
    on=["Monit_MAP_CODE1", "Year"],
    how="inner"
)

water_q_df = water_q_df.with_columns(
    pl.col("Year").cast(pl.Int32)
)

final_summary = parcel_all.join(
    water_q_df,
    left_on=["Monit_MAP_CODE1", "Year"],
    right_on=["DNR_ID_Site_Number", "Year"],
    how="inner"
)

print("parcel_all head:")
print(parcel_all.head())

print("\nwater_q_df head:")
print(water_q_df.head())

print("\nfinal_summary head:")
print(final_summary.head())

print("\nfinal_summary shape (rows, cols):")
print(final_summary.shape)

print("\nfinal_summary head:")
print(final_summary.head())


final_summary.write_csv(
    "data/lakes_processed/water_quality_and_parcel_summaries_2004_to_2015.csv"
)


parcel_all head:
shape: (5, 137)
┌─────────────┬──────┬─────────────┬─────────────┬───┬─────────────┬─────┬────────────┬────────────┐
│ Monit_MAP_C ┆ Year ┆ mean_FIN_SQ ┆ median_FIN_ ┆ … ┆ MINI-MART   ┆ D   ┆ INDUSTRIAL ┆ RELOCATABL │
│ ODE1        ┆ ---  ┆ _FT         ┆ SQ_FT       ┆   ┆ CONVENIENCE ┆ --- ┆ S LIGHT    ┆ E          │
│ ---         ┆ i32  ┆ ---         ┆ ---         ┆   ┆ STORE W*    ┆ i64 ┆ MFTG.      ┆ CLASSROOM  │
│ str         ┆      ┆ f64         ┆ f64         ┆   ┆ ---         ┆     ┆ ---        ┆ ---        │
│             ┆      ┆             ┆             ┆   ┆ i64         ┆     ┆ i64        ┆ i64        │
╞═════════════╪══════╪═════════════╪═════════════╪═══╪═════════════╪═════╪════════════╪════════════╡
│ 02000500-01 ┆ 2012 ┆ 1811.576636 ┆ 1228.0      ┆ … ┆ 0           ┆ 0   ┆ 0          ┆ 0          │
│ 19034800-01 ┆ 2007 ┆ 1633.298962 ┆ 1612.0      ┆ … ┆ 0           ┆ 0   ┆ 0          ┆ 0          │
│ 27071100-01 ┆ 2011 ┆ 935.985793  ┆ 1008.0      ┆ … ┆ 0  